In [1]:
import pandas as pd

In [10]:
def get_query(site_id, variant_code, interval, category_code):
    query = f"""
    DECLARE 
        @SiteId       UNIQUEIDENTIFIER = '{site_id}',
        @VariantCode  NVARCHAR(12)     = '{variant_code}',
        @Interval     NVARCHAR(50)     = '{interval}',
        @CategoryCode NVARCHAR(50)     = '{category_code}';

    WITH ProvisionedTimes AS (
        SELECT
            DATETRUNC(WEEK, MIN([From]))                      AS Starting,
            DATEADD(WEEK, 1, DATETRUNC(WEEK, MAX([To])))      AS Ending
        FROM [PollenSenseLive].[dbo].[AllProvisions] AP
        WHERE AP.SiteID = @SiteId
    ),
    provisionedHours AS (
        SELECT
            Starting,
            Ending,
            @SiteId       AS SiteId,
            @VariantCode  AS VariantCode,
            @Interval     AS [Interval],
            @CategoryCode AS CategoryCode
        FROM ProvisionedTimes

        UNION ALL

        SELECT
            CASE 
                WHEN @Interval = 'hour' THEN DATEADD(hour, 1, Starting)
                ELSE DATEADD(day,  1, Starting)
            END     AS Starting,
            Ending,
            SiteId,
            VariantCode,
            [Interval],
            CategoryCode
        FROM provisionedHours
        WHERE Starting < Ending
    ),
    FoundProvisionedHours AS (
        SELECT
            PH.SiteId,
            PH.VariantCode,
            PH.Starting,
            PH.CategoryCode,
            PH.[Interval],
            VR.[Count],
            VR.PPM3,
            CASE 
                WHEN EXISTS (
                    SELECT 1
                    FROM [PollenSenseLive].[dbo].[VariantRollup] VR2
                    WHERE VR2.Starting     = PH.Starting
                    AND VR2.SiteId       = PH.SiteId
                    AND VR2.VariantCode  = PH.VariantCode
                    AND VR2.[Interval]   = PH.[Interval]
                    
                ) THEN 1
                ELSE 0
            END AS IsRollup
        FROM provisionedHours PH
        LEFT JOIN [PollenSenseLive].[dbo].[VariantRollup] VR
        ON VR.Starting      = PH.Starting
        AND VR.SiteId        = PH.SiteId
        AND VR.VariantCode   = PH.VariantCode
        AND VR.[Interval]    = PH.[Interval]
        AND VR.CategoryCode  = PH.CategoryCode
    )
    SELECT
        SiteId,
        VariantCode,
        Starting,
        [Interval],
        CategoryCode,
        ISNULL([Count], 0) AS [Count],
        ISNULL(PPM3,     0) AS PPM3
    FROM FoundProvisionedHours FPH
    WHERE FPH.IsRollup = 1
    OPTION (MAXRECURSION 0);
"""
    return query



In [22]:
from dotenv import load_dotenv
load_dotenv()
import os
server = os.getenv("DB_SERVER")
username = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")
import urllib.parse
encoded_password = urllib.parse.quote_plus(password)
database = os.getenv("DB_NAME")
conn_str = (
            f"mssql+pyodbc://{username}:{encoded_password}@{server}/{database}"
            "?driver=ODBC+Driver+17+for+SQL+Server"
            "&TrustServerCertificate=yes"
            "&Connection+Timeout=30"
            "&Encrypt=yes"
        )

In [23]:
from sqlalchemy import create_engine
engine = create_engine(conn_str)

In [27]:
query = get_query(
                "A73E57F4-0EED-4AA9-859F-B06CB084FA37", "GS2", "hour", "LOL"
            )

In [34]:
query = f"""
            SELECT *
            FROM [PollenSenseLive].[dbo].[VariantRollup]
            WHERE CategoryCode = 'LOL'
            AND Interval = 'hour'
            AND VariantCode = 'M'
            
        """

In [35]:

df = pd.read_sql(query, engine)

In [36]:
df

,SiteId,VariantCode,Interval,Starting,CategoryCode,Count,PPM3,AverageProbability,FrameCount
0,B6478AB7-4C53-4BA0-A3A6-0660E5E98992,M,hour,2020-05-10 23:00:00,LOL,0.977,16.027606,0.527715,1
1,B6478AB7-4C53-4BA0-A3A6-0660E5E98992,M,hour,2020-05-11 00:00:00,LOL,0.430,7.148594,0.180794,1
2,B6478AB7-4C53-4BA0-A3A6-0660E5E98992,M,hour,2020-05-11 01:00:00,LOL,1.529,25.873614,0.495568,1
3,B6478AB7-4C53-4BA0-A3A6-0660E5E98992,M,hour,2020-05-11 02:00:00,LOL,0.912,15.265408,0.514583,1
4,B6478AB7-4C53-4BA0-A3A6-0660E5E98992,M,hour,2020-05-11 03:00:00,LOL,0.912,15.070032,0.702929,1
...,...,...,...,...,...,...,...,...,...
271,653C97A9-ACC9-4A11-A4F3-25721C89DCF6,M,hour,2020-05-09 06:00:00,LOL,0.558,9.020933,0.505769,1
272,6EE09179-1D94-4229-AE95-FEFEED5297D9,M,hour,2020-12-25 04:00:00,LOL,0.758,11.980536,0.195562,1
273,6EE09179-1D94-4229-AE95-FEFEED5297D9,M,hour,2020-12-25 05:00:00,LOL,0.191,2.964437,0.195562,1
274,0E01830D-1E2E-47C9-90AE-16BE0E7415AA,M,hour,2021-02-04 04:00:00,LOL,0.964,14.998512,0.980262,1


In [17]:
df_new

,SiteId,VariantCode,Starting,Interval,CategoryCode,Count,PPM3,Timezone
1984,91704A9F-2C4F-439E-8172-01E5335B3253,M,2020-04-30 02:00:00,hour,CHE-AMA,0.986,15.588721,America/Chicago
1985,91704A9F-2C4F-439E-8172-01E5335B3253,M,2020-04-30 06:00:00,hour,CHE-AMA,0.988,16.318138,America/Chicago
1986,91704A9F-2C4F-439E-8172-01E5335B3253,M,2020-04-30 12:00:00,hour,CHE-AMA,0.998,15.938854,America/Chicago
1987,91704A9F-2C4F-439E-8172-01E5335B3253,M,2020-04-30 21:00:00,hour,CHE-AMA,0.918,15.283106,America/Chicago
1988,91704A9F-2C4F-439E-8172-01E5335B3253,M,2020-05-01 17:00:00,hour,CHE-AMA,0.910,15.009210,America/Chicago
...,...,...,...,...,...,...,...,...
95354,117392A0-75B7-422F-ADAE-FF1850AA8C8E,M,2020-11-18 01:00:00,hour,CHE-AMA,0.972,16.765293,America/New_York
95355,117392A0-75B7-422F-ADAE-FF1850AA8C8E,M,2020-11-18 16:00:00,hour,CHE-AMA,0.936,16.021774,America/New_York
95356,117392A0-75B7-422F-ADAE-FF1850AA8C8E,M,2020-11-18 18:00:00,hour,CHE-AMA,0.920,17.178704,America/New_York
95357,117392A0-75B7-422F-ADAE-FF1850AA8C8E,M,2020-11-18 21:00:00,hour,CHE-AMA,0.986,16.877971,America/New_York


In [17]:
# Analysis accumulated precip (always zero everywhere)
from herbie import Herbie
from herbie.toolbox import EasyMap, pc

h00 = Herbie("2021-04-28", fxx=0, model="gfs",
    product="pgrb2b.0p25", .xarray(":APCP:")

# Accumulated precip
# Note the different search syntax needed to get the 1hr precip versus the 6hr precip

# Accumulation for 1 hour
h06_1hr = Herbie("2021-04-28", fxx=6).xarray(":APCP:.*:(?:0-1|[1-9]\d*-\d+) hour")

# Accumulation since initialization time
h06_6hr = Herbie("2021-04-28", fxx=6).xarray(":APCP:surface:0-[1-9]*")


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Apr-28 00:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws


c:\Users\sgnka\anaconda3\envs\pollen\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Apr-28 00:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws


c:\Users\sgnka\anaconda3\envs\pollen\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2021-Apr-28 00:00 UTC F06 ┊ GRIB2 @ aws ┊ IDX @ aws


c:\Users\sgnka\anaconda3\envs\pollen\Lib\site-packages\cfgrib\xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


In [15]:
ds

<xarray.Dataset> Size: 25MB
Dimensions:              (step: 6, latitude: 721, longitude: 1440)
Coordinates:
    time                 datetime64[ns] 8B 2021-04-28
  * step                 (step) timedelta64[ns] 48B 01:00:00 ... 06:00:00
    surface              float64 8B 0.0
  * latitude             (latitude) float64 6kB 90.0 89.75 89.5 ... -89.75 -90.0
  * longitude            (longitude) float64 12kB 0.0 0.25 0.5 ... 359.5 359.8
    valid_time           (step) datetime64[ns] 48B 2021-04-28T01:00:00 ... 20...
    gribfile_projection  object 8B None
Data variables:
    tp                   (step, latitude, longitude) float32 25MB 0.0 ... 0.0625
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   gfs
    product:                 pgrb2.0p25
    description:             NOAA Global Forecast System (GFS)
    search:                  :APCP:surface:

In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('F:\Forecasting_PS\Training_prep\GS_data\POL\POL_df.csv')

In [4]:
df

,SiteId,VariantCode,Starting,Interval,CategoryCode,Count,PPM3,Timezone
0,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,GS2,2025-03-11 15:00:00,hour,POL,0.505,6.938501,America/Los_Angeles
1,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,GS2,2025-03-11 16:00:00,hour,POL,0.482,4.584916,America/Los_Angeles
2,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,GS2,2025-03-11 17:00:00,hour,POL,0.000,0.000000,America/Los_Angeles
3,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,GS2,2025-03-11 18:00:00,hour,POL,0.000,0.000000,America/Los_Angeles
4,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,GS2,2025-03-11 19:00:00,hour,POL,0.000,0.000000,America/Los_Angeles
...,...,...,...,...,...,...,...,...
1038219,117392A0-75B7-422F-ADAE-FF1850AA8C8E,GS2,2023-12-07 09:00:00,hour,POL,0.000,0.000000,America/New_York
1038220,117392A0-75B7-422F-ADAE-FF1850AA8C8E,GS2,2023-12-07 10:00:00,hour,POL,0.000,0.000000,America/New_York
1038221,117392A0-75B7-422F-ADAE-FF1850AA8C8E,GS2,2023-12-07 11:00:00,hour,POL,0.000,0.000000,America/New_York
1038222,117392A0-75B7-422F-ADAE-FF1850AA8C8E,GS2,2023-12-07 12:00:00,hour,POL,0.982,5.158883,America/New_York


In [5]:
df = pd.read_csv('F:\Pollensense\Clean_codes\data_all\POL\POL_df.csv')
df

,SiteId,VariantCode,Interval,Starting,CategoryCode,Count,PPM3,AverageProbability,FrameCount,Timezone
0,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,M,hour,2025-03-11 15:00:00,POL,0.505,6.938501,0.438278,1,America/Los_Angeles
1,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,M,hour,2025-03-11 16:00:00,POL,0.482,4.584916,0.438278,1,America/Los_Angeles
2,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,M,hour,2025-03-12 22:00:00,POL,0.358,7.123765,0.562235,1,America/Los_Angeles
3,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,M,hour,2025-03-12 23:00:00,POL,0.616,10.743678,0.562235,1,America/Los_Angeles
4,29AECC08-40DF-4FD4-9F6C-39D5BCFA7723,M,hour,2025-03-13 15:00:00,POL,0.200,3.106184,0.645386,1,America/Los_Angeles
...,...,...,...,...,...,...,...,...,...,...
730269,94C8B5DA-6EFD-4653-BD32-E2EA1FF33F09,M,hour,2021-10-18 03:00:00,POL,0.904,17.459781,0.676725,1,America/Chicago
730270,94C8B5DA-6EFD-4653-BD32-E2EA1FF33F09,M,hour,2021-10-18 04:00:00,POL,2.462,45.780361,0.597003,1,America/Chicago
730271,94C8B5DA-6EFD-4653-BD32-E2EA1FF33F09,M,hour,2021-10-18 05:00:00,POL,1.453,27.223137,0.619344,1,America/Chicago
730272,94C8B5DA-6EFD-4653-BD32-E2EA1FF33F09,M,hour,2021-10-18 06:00:00,POL,0.859,15.677472,0.408350,1,America/Chicago
